In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
import os
from PIL import Image

from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import datasets, transforms

import pytorch_lightning as pl
from pytorch_lightning import LightningModule
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

from torchmetrics.classification import MulticlassAccuracy, F1Score

import pytorch_optimizer as optim1
import optuna

pl.seed_everything(42)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Seed set to 42


In [2]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

class KaggleTestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Path ke direktori berisi semua gambar test.
            transform (callable, optional): Transformasi yang akan diterapkan pada gambar.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = sorted([f for f in os.listdir(root_dir) if os.path.isfile(os.path.join(root_dir, f))])

    def __len__(self):
        """Mengembalikan jumlah total gambar dalam dataset."""
        return len(self.image_files)

    def __getitem__(self, idx):
        """
        Mengambil satu item data.

        Args:
            idx (int): Indeks dari item.
        
        Returns:
            tuple: (image, image_name) di mana image_name adalah nama file.
        """
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        image_name = self.image_files[idx]
        return image, image_name

In [ ]:
train= transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.RandomPerspective(),
            transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
            transforms.ColorJitter(),
            transforms.Grayscale(num_output_channels=3),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
val = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_dir = 'dsc-logika-ui-2025/Train'

train_dataset = datasets.ImageFolder(root=f'{data_dir}/Train', transform=train)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
print(f'Jumlah data test: {len(train_dataset)}')

Jumlah data test: 1752


In [ ]:
data_test_dir = 'dsc-logika-ui-2025/Test/Test'

test_dataset = KaggleTestDataset(root_dir=data_test_dir, transform=val)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)
print(f'Jumlah data test: {len(test_dataset)}')

Jumlah data test: 444


In [5]:
feature, target = next(iter(train_loader))
feature.shape, target.shape

(torch.Size([32, 3, 224, 224]), torch.Size([32]))

In [6]:
label2cat, idxclass = train_dataset.class_to_idx, train_dataset.classes
label2cat

{'balinese': 0, 'batak': 1, 'dayak': 2, 'javanese': 3, 'minangkabau': 4}

Arsitektur dan config

In [7]:
def conv_block(in_feature, out_feature, padding=1, stride=1,
             activation="relu", pool =True, maxpool=True, kernel_size=3,
             kernel_size_pool=2, pool_stride=2)-> list[nn.Sequential]:
    layers = [nn.Conv2d(in_feature, out_feature, kernel_size=kernel_size, padding=padding, stride=stride)]
    if activation == "relu":
        layers.append(nn.ReLU())
    elif activation == "leakyrelu":
        layers.append(nn.LeakyReLU())
    elif activation == "sigmoid":
        layers.append(nn.Sigmoid())
    elif activation == 'mish': layers.append(nn.Mish())
    elif activation == "tanh":
        layers.append(nn.Tanh())
    if pool:
        if maxpool:
            layers.append(nn.MaxPool2d(kernel_size=kernel_size_pool, stride=pool_stride))
        else:
            layers.append(nn.AvgPool2d(kernel_size=kernel_size_pool, stride=pool_stride))
    else:
        layers.append(nn.Identity())
    return nn.Sequential(*layers)


def linear_block(in_features, out_features, activation=None, dropout=0.0, batch_norm=None):
    layers = [nn.Linear(in_features, out_features)]
    if batch_norm:
        layers.append(nn.BatchNorm1d(out_features))
    if activation == 'relu':
        layers.append(nn.ReLU())
    elif activation == 'sigmoid':
        layers.append(nn.Sigmoid())
    elif activation == 'tanh':
        layers.append(nn.Tanh())
    elif activation == 'leakyrelu':
        layers.append(nn.LeakyReLU())
    elif activation == 'mish': layers.append(nn.Mish())
    elif activation == 'softmax':
        layers.append(nn.Softmax(dim=1))
    elif activation == 'elu':
        layers.append(nn.ELU())
    elif activation == 'selu':
        layers.append(nn.SELU())
    elif activation == 'lsoftmax':
        layers.append(nn.LogSoftmax(dim=1))
    if dropout > 0.0:
        layers.append(nn.Dropout(dropout))
    return nn.Sequential(*layers)

In [8]:

class CNN(LightningModule):
    def __init__(self, dropout) -> None:
        super().__init__()
        self.conv = nn.Sequential(
            conv_block(3, 8, activation='mish'),
            conv_block(8, 16, activation='mish'),
            conv_block(16, 32, activation='mish'),
            nn.Flatten()
        )
        self.fc = nn.Sequential(
            linear_block(32*784, 256, activation='mish', dropout=dropout, batch_norm=True),
            linear_block(256, 128, activation='mish', dropout=dropout, batch_norm=True),
            linear_block(128, 64, activation='mish', dropout=dropout, batch_norm=True),
            linear_block(64, 5, activation=None)
        )
    def forward(self, X):
        return self.fc(self.conv(X))
        
class PL(LightningModule):
    def __init__(self, model, learning_rate) -> None:
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.criterion = nn.CrossEntropyLoss()
        self.macroF1 = F1Score(num_classes=5, average='macro', task='multiclass')
    
    def forward(self, X):
        return self.model(X)
    
    def _common_step(self, batch, batch_idx):
        X, labels = batch
        outputs = self(X) 
        loss = self.criterion(outputs, labels)
        macrof1 = self.macroF1(outputs, labels)
        return loss, macrof1

    def training_step(self, batch, batch_idx):
        loss, macroF1 = self._common_step(batch, batch_idx)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_macrof1', macroF1, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, macroF1 = self._common_step(batch, batch_idx)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_macrof1', macroF1, on_epoch=True, prog_bar=True, logger=True)

    def test_step(self, batch, batch_idx):
        loss, macroF1 = self._common_step(batch, batch_idx)
        self.log('test_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('test_macrof1', macroF1, on_epoch=True, prog_bar=True, logger=True)

    def backward(self, loss, *args, **kwargs):
        loss.backward(create_graph=True)

    def configure_optimizers(self):
        optimizer = optim1.Lion(self.parameters(), lr=self.learning_rate)
        return optimizer

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        """
        Langkah prediksi untuk satu batch data test.
        """
        images, image_names = batch
        outputs = self.forward(images)
        _, predicted_labels = torch.max(outputs, 1)
        return {"image_names": image_names, "preds": predicted_labels}

In [ ]:
if torch.cuda.is_available():
    accelerator_type = 'gpu'
    devices_to_use = 1
else:
    accelerator_type = 'cpu'
    devices_to_use = 'auto'

checkpoint_callback = ModelCheckpoint(
    monitor='train_macrof1',
    dirpath='checkpoints/',
    filename='logikaui-{epoch:02d}-{train_macrof1:.2f}',
    save_top_k=1,
    mode='max'
)
early_stopping = EarlyStopping(
    monitor='train_loss',
    patience=10,
    mode='min',
)
lr_monitor_callback = LearningRateMonitor(logging_interval='epoch')

trainer1 = pl.Trainer(
    max_epochs=300,
    callbacks=[checkpoint_callback, early_stopping, lr_monitor_callback],
    logger=TensorBoardLogger("tb_logs", name="simple_model_experiment"),
    accelerator=accelerator_type,
    devices=devices_to_use,
    log_every_n_steps=10,
    deterministic=True,
    precision='16-mixed'
    
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


## Train

In [ ]:
model = PL(CNN(dropout=0.2), learning_rate=1e-3)
model = torch.compile(model) 

In [11]:
trainer1.fit(model, train_loader, val_dataloaders=None, )


  | Name      | Type              | Params | Mode 
--------------------------------------------------------
0 | model     | CNN               | 6.5 M  | train
1 | criterion | CrossEntropyLoss  | 0      | train
2 | macroF1   | MulticlassF1Score | 0      | train
--------------------------------------------------------
6.5 M     Trainable params
0         Non-trainable params
6.5 M     Total params
25.885    Total estimated model params size (MB)
35        Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

In [ ]:
if __name__ == "__main__":
    preds = trainer1.predict(model, test_loader, ckpt_path='best')

In [ ]:
predictions = []
for batch_result in preds:
    # batch_result adalah dictionary yang kita return dari predict_step
    image_names = batch_result['image_names']
    preds = batch_result['preds'].cpu().numpy() # Pindahkan ke CPU
    
    for name, label in zip(image_names, preds):
        predictions.append({
            'id': name,      # Nama kolom sesuai standar Kaggle
            'style': label   # Nama kolom sesuai standar Kaggle
        })